In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [3]:
# Tinggi dan lebar untuk mengubah ukuran frame video
IMAGE_HEIGHT_MOBILENET, IMAGE_WIDTH_MOBILENET = 224, 224
IMAGE_HEIGHT_MOVINET, IMAGE_WIDTH_MOVINET = 172, 172
# Jumlah frame yang akan diambil dari setiap video
SEQUENCE_LENGTH_MOBILENET = 16
SEQUENCE_LENGTH_MOVINET = 50
# Direktori tempat dataset disimpan
DATASET_DIR = "../data/"
# Daftar class untuk klasifikasi
CLASSES_LIST = ["NonViolence", "Violence"]

In [4]:
def extract_frames(video_path, sequence_length=SEQUENCE_LENGTH_MOBILENET, image_height=IMAGE_HEIGHT_MOBILENET, image_width=IMAGE_WIDTH_MOBILENET):
    cap = cv2.VideoCapture(video_path)

    # 1. Dapatkan total frame
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # 2. Tentukan indeks frame yang akan diambil, sampling merata
    if total_frames < sequence_length:
        # Kalau video terlalu pendek, ambil semua dan ulangi terakhir sampai seq_len
        indices = list(range(total_frames)) + [total_frames - 1] * (sequence_length - total_frames)
    else:
        step = total_frames / sequence_length
        indices = [int(i * step) for i in range(sequence_length)]

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            # kalau gagal baca frame, ulangi frame terakhir
            frame = frames[-1] if frames else np.zeros((image_height, image_width, 3), np.uint8)
        # 3. Center‐crop ke square
        h, w = frame.shape[:2]
        m = min(h, w)
        y0 = (h - m) // 2
        x0 = (w - m) // 2
        crop = frame[y0:y0+m, x0:x0+m]
        # 4. Resize, RGB, normalisasi
        crop = cv2.resize(crop, (image_height, image_width), interpolation=cv2.INTER_AREA)
        rgb  = cv2.cvtColor(crop, cv2.COLOR_BGR2RGB)
        frames.append(rgb.astype(np.float32) / 255.0)

    cap.release()
    return np.stack(frames, axis=0)

In [5]:
def create_dataset(sequence_length=SEQUENCE_LENGTH_MOBILENET, image_height=IMAGE_HEIGHT_MOBILENET, image_width=IMAGE_WIDTH_MOBILENET):
    features = []
    labels = []
    video_files_paths = []

    # 1. Iterasi melalui setiap kelas
    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        
        # 2. Dapatkan daftar file video dalam direktori kelas
        class_dir = os.path.join(DATASET_DIR, class_name)
        files_list = os.listdir(class_dir)

        # 3. Iterasi melalui setiap file video
        for file_name in files_list:
            # a. Bangun path lengkap ke file video
            video_file_path = os.path.join(class_dir, file_name)

            # b. Ekstrak frame dari video
            frames = extract_frames(video_file_path, sequence_length, image_height, image_width)

            # c. Periksa apakah jumlah frame sesuai dengan SEQUENCE_LENGTH
            if len(frames) == sequence_length:
                # d. Simpan data ke dalam list yang sesuai
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)

    # 4. Konversi list ke array NumPy
    features = np.asarray(features)
    labels = np.array(labels)

    # 5. Kembalikan dataset
    return features, labels, video_files_paths


In [5]:
# Buat dataset
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: NonViolence
Extracting Data of Class: Violence


In [6]:
# Konversi label ke one hot encoded format
one_hot_encoded_labels = to_categorical(labels)

In [7]:
# Split data menjadi 90% data train dan 10% data test.
X_train, X_test, y_train, y_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1, shuffle = True, random_state = 42)


In [8]:
print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

(1800, 16, 224, 224, 3) (1800, 2)
(200, 16, 224, 224, 3) (200, 2)


In [9]:
# Saving the extracted data
os.makedirs("../data/ProcessedData/", exist_ok=True)
np.save("../data/ProcessedData/X_train.npy", X_train)
np.save("../data/ProcessedData/y_train.npy", y_train)
np.save("../data/ProcessedData/X_test.npy", X_test)
np.save("../data/ProcessedData/y_test.npy", y_test)
print("Data has been saved successfully.")

Data has been saved successfully.


# MoViNet Data Preparation

In [6]:
# Buat dataset
features, labels, video_files_paths = create_dataset(SEQUENCE_LENGTH_MOVINET, IMAGE_HEIGHT_MOVINET, IMAGE_WIDTH_MOVINET)

Extracting Data of Class: NonViolence
Extracting Data of Class: Violence


In [7]:
# Konversi label ke one hot encoded format
one_hot_encoded_labels = to_categorical(labels)

In [8]:
# Split data menjadi 90% data train dan 10% data test.
X_train, X_test, y_train, y_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.1, shuffle = True, random_state = 42)


In [9]:
print(X_train.shape,y_train.shape)
print(X_test.shape, y_test.shape)

(1800, 50, 172, 172, 3) (1800, 2)
(200, 50, 172, 172, 3) (200, 2)


In [10]:
AUTOTUNE   = tf.data.AUTOTUNE
BATCH_SIZE = 8

def make_tf_dataset(X, y, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X))
    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

train_ds = make_tf_dataset(X_train, y_train, shuffle=True)
val_ds   = make_tf_dataset(X_test, y_test, shuffle=False)

In [ ]:
# Saving the extracted data
os.makedirs("../data/ProcessedDataMoViNet/", exist_ok=True)
np.save("../data/ProcessedDataMoViNet/X_train.npy", X_train)
np.save("../data/ProcessedDataMoViNet/y_train.npy", y_train)
np.save("../data/ProcessedDataMoViNet/X_test.npy", X_test)
np.save("../data/ProcessedDataMoViNet/y_test.npy", y_test)
print("Data has been saved successfully.")

Data has been saved successfully.
